In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split

# Inicializa uma sessão do Spark
spark = SparkSession.builder \
    .appName("Exemplo") \
    .getOrCreate()
    
file = '/workspaces/api-geral/CAR/dados/ford/modelo_ford.txt'  

df = spark.read.option("delimiter", ";", ).option("header", "true").csv(file)

# Exibe o DataFrame
df.show(truncate=False)
# df.show()


+------+--------------------------------------------------------------------+
|codigo|modelo                                                              |
+------+--------------------------------------------------------------------+
|655   |{'codigo': 655, 'nome': 'Aerostar Mini-Van 3.8'}                    |
|656   |{'codigo': 656, 'nome': 'Aspire 1.3'}                               |
|657   |{'codigo': 657, 'nome': 'Belina GL 1.8 / 1.6'}                      |
|658   |{'codigo': 658, 'nome': 'Belina L 1.8/ 1.6'}                        |
|9452  |{'codigo': 9452, 'nome': 'Bronco Sport Wildtrak 2.0 Tb 16V AWD Aut'}|
|659   |{'codigo': 659, 'nome': 'Club Wagon V8'}                            |
|660   |{'codigo': 660, 'nome': 'Club Wagon XLT 4.9 V6'}                    |
|661   |{'codigo': 661, 'nome': 'Contour SE/ GL/ LX  2.0 16V'}              |
|662   |{'codigo': 662, 'nome': 'Contour SE/GL/ LX 2.5 24V'}                |
|663   |{'codigo': 663, 'nome': 'Corcel II GL/GT'}              

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import requests

# Inicializa uma sessão do Spark
spark = SparkSession.builder \
    .appName("Exemplo") \
    .getOrCreate()

# Solicita a lista de modelos
url_modelos = 'https://parallelum.com.br/fipe/api/v1/carros/marcas/22/modelos'
response = requests.get(url_modelos)
dados_modelos = response.json()

data_list = []

if dados_modelos:
    modelos = dados_modelos['modelos']
    for modelo in modelos:
        codigo = modelo['codigo']
        nome = modelo['nome']
        data_list.append({
            'codigo': codigo,
            'modelo': nome
        })

# Inicializa a lista de valores fora dos loops principais
lista_ano_valor = []

# Itera sobre cada modelo
for modelo in data_list:
    codigo_modelo = modelo['codigo']
    
    # Solicita a lista de anos para o modelo atual
    url_anos = f'https://parallelum.com.br/fipe/api/v1/carros/marcas/22/modelos/{codigo_modelo}/anos'
    response = requests.get(url_anos)
    dados_anos = response.json()
    
    if dados_anos:
        anos = [item['codigo'] for item in dados_anos]
        
        # Itera sobre cada ano
        for ano in anos:
            # Solicita as informações de valor para o modelo e ano atual
            url_valor = f'https://parallelum.com.br/fipe/api/v1/carros/marcas/22/modelos/{codigo_modelo}/anos/{ano}'
            response = requests.get(url_valor)
            dados_valor = response.json()
            
            # Adiciona os dados à lista de valores
            if response.status_code == 200:
                tipo = dados_valor['TipoVeiculo']
                vlr = dados_valor['Valor']
                marc = dados_valor['Marca']
                mod = dados_valor['Modelo']
                anMod = dados_valor['AnoModelo']
                comb = dados_valor['Combustivel']
                codf = dados_valor['CodigoFipe']
                mRef = dados_valor['MesReferencia']
                sglaComb = dados_valor['SiglaCombustivel']
                
                lista_ano_valor.append({
                    'tipo_veiculo': tipo,
                    'valor': vlr,
                    'marca': marc,
                    'modelo': mod,
                    'anoModelo': anMod,
                    'combustivel': comb,
                    'codigoFipe': codf,
                    'mes_referencia': mRef,
                    'sigla_combustivel': sglaComb,
                    'Ano': ano
                })

# Cria um DataFrame Spark a partir da lista de valores
df = spark.createDataFrame(lista_ano_valor)

# Exibe o DataFrame
df.show()

# Encerra a sessão Spark
spark.stop()


24/03/23 15:28:22 WARN Utils: Your hostname, codespaces-a33db6 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/03/23 15:28:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/23 15:28:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/23 15:28:37 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


KeyboardInterrupt: 